In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
data = pd.read_csv('../rawdata/SFPD_Incidents_-_from_1_January_2003.csv')
data.head(3)

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,160213624,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,03/13/2016,23:50,CENTRAL,NONE,VANNESS AV / NORTHPOINT ST,-122.425301,37.805077,"(37.8050769933248, -122.425300930134)",16021362403074
1,160213599,ASSAULT,BATTERY,Sunday,03/13/2016,23:48,NORTHERN,NONE,1800 Block of FILBERT ST,-122.429951,37.798805,"(37.7988051605065, -122.429951447549)",16021359904134
2,160213599,ASSAULT,AGGRAVATED ASSAULT WITH A KNIFE,Sunday,03/13/2016,23:48,NORTHERN,NONE,1800 Block of FILBERT ST,-122.429951,37.798805,"(37.7988051605065, -122.429951447549)",16021359904012


In [3]:
robbery_street = data[(data.Category == 'ROBBERY') & data.Descript.str.contains("ON THE STREET") ]
robbery_street.head(6)

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
18,160213527,ROBBERY,"ROBBERY ON THE STREET, STRONGARM",Sunday,03/13/2016,22:40,MISSION,NONE,3400 Block of CESAR CHAVEZ ST,-122.419269,37.748032,"(37.7480322172379, -122.419268840041)",16021352703014
284,160211496,ROBBERY,ROBBERY ON THE STREET WITH A DANGEROUS WEAPON,Saturday,03/12/2016,23:04,INGLESIDE,NONE,100 Block of NEY ST,-122.427180,37.730025,"(37.730024796823, -122.427180210853)",16021149603013
543,160219224,ROBBERY,"ROBBERY ON THE STREET, STRONGARM",Saturday,03/12/2016,02:55,INGLESIDE,NONE,3200 Block of MISSION ST,-122.420526,37.744725,"(37.7447254713657, -122.420526309078)",16021922403014
574,160215608,ROBBERY,"ROBBERY ON THE STREET, STRONGARM",Saturday,03/12/2016,01:00,CENTRAL,NONE,KEARNY ST / CLAY ST,-122.404799,37.794453,"(37.7944525389481, -122.404799424951)",16021560803014
632,160208954,ROBBERY,ROBBERY ON THE STREET WITH A KNIFE,Friday,03/11/2016,21:40,CENTRAL,NONE,STOCKTON ST / PACIFIC AV,-122.408433,37.796704,"(37.7967039278443, -122.408433097286)",16020895403012
917,160206362,ROBBERY,"ROBBERY ON THE STREET, STRONGARM",Friday,03/11/2016,02:00,TENDERLOIN,NONE,TURK ST / LEAVENWORTH ST,-122.414056,37.782793,"(37.7827931071006, -122.414056291891)",16020636203014


In [7]:
robbery_street.shape

(25070, 13)

In [4]:
# http://geoffboeing.com/2015/10/exporting-python-data-geojson/

def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [8]:
cols = ['Date', 'Time', 'Descript']

# Max is 25070
rows_to_file = 25070

geojson = df_to_geojson(robbery_street[:rows_to_file], cols,lat='Y',lon='X')

output_filename = 'robbery-street'+str(rows_to_file)+'.js'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=4) 

In [9]:
# Write to csv

cols = ['Date', 'Time', 'Descript', 'X', 'Y']

# Max is 25070
rows_to_file = 25070

robbery_street.to_csv('robbery-street'+str(rows_to_file)+'.csv',sep='|',index=False,columns = cols)